In [5]:
import requests
from bs4 import BeautifulSoup

domain = "https://help.jp.mercari.com"
top_page_url = f"{domain}/guide/"

In [6]:
categories = []
articles = []

def get_articles_and_categories(link: str):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    tags = soup.find_all("a")
    for tag in tags:
        href = tag.get("href")
        if href.startswith(domain):
            relative_href = href[len(domain):]
        else:
            relative_href = href
#         print(f"href:{relative_href}")
        if href.startswith("/guide/categories/") and href not in categories:
            categories.append(href)
            get_articles_and_categories(f"{domain}{relative_href}")
        elif href.startswith("/guide/articles/") and href not in articles:
            articles.append(href)

get_articles_and_categories(top_page_url)

In [7]:
from llama_index import download_loader

BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
loader = BeautifulSoupWebReader()
documents = loader.load_data(urls=[f"{domain}{articles[0]}"])

In [9]:
from llama_index import GPTVectorStoreIndex, PromptHelper

index = GPTVectorStoreIndex.from_documents(documents)

In [10]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os
import pandas as pd
import tiktoken
import openai
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity

In [14]:
from langchain.chat_models import ChatOpenAI
from llama_index import LLMPredictor,ServiceContext

BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
loader = BeautifulSoupWebReader()
service_context = ServiceContext.from_defaults(llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name="gpt-3.5-turbo")))

documents = loader.load_data(urls=abs_articles)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)


In [15]:
index.storage_context.persist()

In [23]:
def response(message):
    qry = f"""
あなたはメルカリのカスタマーサポートです。与えられたインデックスに関連するお客様の質問の対して、丁寧かつ分かりやすく回答します。```で囲まれた質問に対して、質問者の言語で回答します。必要に応じて、動画やスクリーンキャプチャ画像も追加して回答します。

質問: ```{message}```
回答:
    """
    query_engine = index.as_query_engine()
    print(query_engine.query(qry))

In [24]:
response("出品方法を教えてください。")

出品方法については、以下のURLに詳しい手順が記載されています。ご参照ください。
https://help.jp.mercari.com/guide/articles/54/


In [25]:
response("メルカードを発行後支払い方法がメルカードになってしまい以前のスマート払いにしたいのでやり方を教えてください。")

ご質問ありがとうございます。メルカードから以前のスマート払いに支払い方法を変更する方法についてご案内いたします。

まず、メルカリアプリにログインしてください。次に、画面左上の「マイページ」をタップしてください。画面に表示される「設定」をタップし、次に「支払い方法」をタップしてください。そこで、メルカードの支払い方法を削除し、以前のスマート払いを選択することができます。

上記の手順で、メルカードから以前のスマート払いに支払い方法を変更することができます。もし、ご不明な点がございましたら、お手数ですがカスタマーサポートまでお問い合わせください。


In [26]:
response("メルカード決済ではなく以前のメルペイスマート払いは使えないのですか？")

申し訳ございませんが、現在メルペイスマート払いはご利用いただけません。代わりに、メルカードをお使いいただくことが可能です。何かご不明な点がございましたら、お気軽にお問い合わせください。


In [27]:
response("メルカリポイントとメルカードの2つのみ使用しましたが、このメルカードでの支払いのことを、後払いと仰せなのでしょうか。")

はい、メルカードを使用した支払いには後払いのオプションがあります。メルカードは、メルペイアプリで発行されるプリペイドカードで、メルカリでの取引や、メルカリ以外のお店でも利用できます。後払いオプションを有効にすることで、商品を受け取ってから後に支払いが可能となります。ご利用には、事前にメルカードを登録する必要があります。詳細については、メルカリのヘルプページをご確認ください。必要であれば、動画やスクリーンキャプチャ画像も追加して回答します。


In [28]:
response("""「【メルペイ】3月請求の支払い期限は2023年3月31日(金)です」というタイトルのメールが配信されました。「あと払い2月利用分の支払い期限は2023年3月31日(金)です」とのことですが、メルカリ内での購入において、後払いを利用した記憶がありません。ついては以下2点にご教示願います。
1，メルカリポイントとメルカードの2つのみ使用しましたが、このメルカードでの支払いのことを、後払いと仰せなのでしょうか。
2，この後、どこかへ振込に行くなど、私のほうで必要な対処はありますか
""")

この度はお問い合わせいただきありがとうございます。お送りいただいたメールについて、以下のように回答させていただきます。

1. メールに記載された「あと払い2月利用分の支払い期限は2023年3月31日(金)です」というのは、メルペイのあと払いを利用した場合の支払期限を示しているものです。もしも、ご利用いただいたのがメルカードでの支払いのみであれば、あと払いを利用したわけではありません。 

2. ご利用いただいたのがメルカードであれば、支払いについては自動的に行われるため、追加の対処は必要ありません。ただし、もしも別の支払い方法を利用された場合には、支払い期限までに必ずお支払いいただくようお願いいたします。

以上、ご確認いただけますようお願いいたします。何かご不明な点がございましたら、お気軽にお問い合わせください。


In [29]:
response("この品をメルカードでの支払いで購入したのですが、スマートメル払いの支払い方法を設定せよと連絡が来ています。何故カード決済にならないのでしょうか？何か騙されている感じです。どうしたら良いか連絡ください。")

申し訳ありませんが、メルペイのあと払い（スマートメル払い）には、メルカードでの支払いでも設定が必要になります。この設定をすることで、今後もスムーズにメルペイを利用できるようになります。ご心配な点がある場合は、お手数ですがお問い合わせフォームからカスタマーサポートまでお問い合わせください。


In [30]:
q = """
ポイント欲しさにメルカードを申し込みましたが、また更にポイントをもらうため、メルペイの後払いをするには何をすればいいですか？コンビニなどで普通にID払いで買い物すればいいのでしょうか？メルカードは利子がつくようですが、それだったら普通にメルペイで買い物した方が良いのでは？メルカードを利用する価値がよくわかりません。高額な買い物ができるということなのでしょうか？メルペイの後払いとメルカードの違いがよくわからないです。あとバーチャルカードもありますよね？バーチャルカードとメルカードが別々の意味があるのでしょうか？
"""
response(q)

メルペイの後払いをするためには、まずはメルペイのアプリをダウンロードし、クレジットカード情報を登録する必要があります。その後、メルペイの後払いが利用可能なお店で商品を購入する際に、決済方法として「メルペイの後払い」を選択してください。ID払いは利用できませんのでご注意ください。また、メルカードとメルペイの後払いは異なるサービスです。メルカードはクレジットカードとして利用でき、ポイント還元率も高いですが、利息がかかる場合があります。一方、メルペイの後払いは、商品を受け取ってから支払いができる便利なサービスです。バーチャルカードは、メルカードと同じくクレジットカードとして利用できますが、物理的なカードは発行されず、オンラインショッピングなどで利用することができます。以上、ご参考になれば幸いです。


In [31]:
q = """
メルカードの申し込みをしたのですが、後払いや定額払いの設定はするつもりがなく、申し込み時にも「後払い、定額払いにはなりません」と書かれていたように思うのですが、気づいたら「後払い、定額払い」の項目がでるようになっていました。すでに手数料等がかかっているのでしょうか？
"""
response(q)

申し込み時に「後払い、定額払いにはなりません」と表示されたにもかかわらず、後払いや定額払いの設定ができるようになっていたとのことで、手数料等がかかってしまったようですね。申し訳ありません。後払いや定額払いの設定は、申し込み時に選択しなかった場合でも、後から設定することができるため、設定されてしまったのかもしれません。手数料等については、詳細がわからないため、メルカリの事務局までお問い合わせいただくか、メルカリのサポートページをご確認ください。


In [32]:
q = """
聞いた事もない「あと払い」という払い方は通常のクレジット払いの事と同じか？メルカードを通常のクレジットカードだと思い申込みしたが、後払いで手数料かかるなら要らない。手数料かからない払い方はあるのか？
"""
response(q)

「あと払い」とは通常のクレジット払いとは異なる支払い方法で、商品を受け取ってから後で支払うことができます。メルカードは、メルカリやメルペイでの支払いに特化したカードで、あと払いを利用する場合には手数料がかかります。ただし、定額払いや即時払いなど手数料がかからない支払い方法もあります。詳細については、メルペイのサポートセンターでご確認ください。


In [33]:
response("""
メルペイスマート払いだとメルカリポイントが1％〜4％つくと見ましたが、メルペイスマート払いをして残高で清算してもポイントはつくのでしょうか？
残高でそのまま購入するとメルカリポイントはつかないので、ポイントを沢山つけたい場合はメルペイスマート払いをしてから残高で精算したほうが良いということでしょうか？
""")

はい、メルペイスマート払いをして残高で清算しても、ポイントがつきます。ただし、ポイントを直接購入することはできないため、必ず一度ポイントを購入する必要があります。また、ポイントを沢山つけたい場合は、メルペイスマート払いをしてから残高で精算する方が良いです。詳細については、関連ガイドをご確認ください。動画やスクリーンキャプチャ画像が必要であれば、お知らせください。


In [34]:
response("""商品を出品して売れたお金(残高)で購入したらポイントはつくんでしょうか？ 
売れたお金(残高)で購入するより、残高を口座に振替して、メルカード払いで購入した方がお得なんでしょうか？
""")

「売れたお金(残高)で購入するより、残高を口座に振替して、メルカード払いで購入した方がお得なんでしょうか？」とのご質問ですね。商品を出品して売れたお金(残高)でポイントを購入することはできません。必ず一度ポイントを購入していただく必要があります。また、残高を口座に振替して、メルカード払いで購入することもできますが、ポイントを購入する場合に比べるとポイントがもらえる割合が低くなってしまいます。ご不明点があれば、いつでもお問い合わせください。


In [35]:
response("""メルカードは、紐付けした後に普通に売上金で購入してもポイントは付かないですか？
ポイントが付くのは後払い、定額払いなど限定ですか？
では後払いや定額払い以外で、普段売り上げ金で購入している者にはあまり有効なカードではないという見解で宜しいでしょうか？普段のメルカリの買い物を売り上げ金での購入だと1%〜4%のポイントが付かないという事ですよね？
""")

申し訳ありませんが、メルカードでの売上金での購入はポイントが付きません。ポイントが付くのは後払いや定額払いなど限定です。普段のメルカリの買い物を売上金での購入だと1%〜4%のポイントが付かないということは正しいです。ただし、ポイントが必要な場合は、売上金をポイントに換金することができます。ご了承ください。


In [36]:
response("""定額払いを解約したいのですが、どこを探しても解約ページが見つかりません。
""")

解約ページにアクセスするためには、ログイン後に「マイページ」に進み、「設定」から「定額払い」を選択してください。そこから解約手続きを進めることができます。もし見つからない場合は、カスタマーサポートにお問い合わせください。
